In [32]:
!pip install fiona

In [33]:
## EDIT To save in save in safe folder first then move
## Dir creating on loop reading the file
# run 90 km 95 pct
# prep to run 3-month composite

In [34]:
import os
# hop to a local, healthy path (not on /projects or other EFS mounts)
os.chdir("/tmp")

In [35]:
## Changes
## Move directory creation to running composite loop
## Save into "Safe" directory and then move

In [36]:
import os
import subprocess

from pathlib import Path
from datetime import datetime
from calendar import monthrange

import fiona

import multiprocessing as mp

import geopandas as gpd

import copy

In [37]:
import sys
sys.path.append('/projects/my-private-bucket/code-git-shared/icesat2_boreal/lib')
import plotlib
from mosaiclib import *

In [38]:
import rasterio
from rasterio.plot import show_hist, show
import numpy as np
import matplotlib.pyplot as plt

In [39]:
import multiprocessing as mp

num_cores = mp.cpu_count()
print(f"Number of CPU cores: {num_cores}")

import psutil

mem = psutil.virtual_memory()
print(f"Total memory: {mem.total / 1e9:.2f} GB")
print(f"Available memory: {mem.available / 1e9:.2f} GB")

Number of CPU cores: 32
Total memory: 267.33 GB
Available memory: 237.36 GB


In [40]:
from pathlib import Path

In [41]:
# --- minimal imports here so you can paste this as-is ---
from pathlib import Path
import os, shutil

In [42]:
# Functions 
def wrapper_composite(params):

    FOCAL_TILE = params.get('FOCAL_TILE')
    SAT_API = params.get('SAT_API')
    MS_COMP_TYPE = params.get('MS_COMP_TYPE')
    YEAR = params.get('YEAR')
    MIN_N_FILT_RESULTS = params.get('MIN_N_FILT_RESULTS')
    SEASON_START = params.get('SEASON_START')
    SEASON_STOP = params.get('SEASON_STOP')
    # INDEX_FN = params.get('INDEX_FN')
    # INDEX_LYR = params.get('INDEX_LYR')
    
    STAT = params.get('STAT')
    STAT_PCT = params.get('STAT_PCT')
    TARGET_SPECTRAL = params.get('TARGET_SPECTRAL')

    INDEX_FN =  params.get('INDEX_FN') #'https://maap-ops-workspace.s3.amazonaws.com/shared/montesano/databank/boreal_tiles_v004.gpkg'
    INDEX_LYR = params.get('INDEX_LYR') # 'boreal_tiles_v004'
    
    YEAR_START, YEAR_STOP = (YEAR, YEAR)
    HLS_PRODUCT = params.get('HLS_PRODUCT') #HLS_PRODUCT = 'H30'
    MAX_CLOUDS = params.get('MAX_CLOUDS') #MAX_CLOUDS = 0

    OUTDIR = params.get('OUTDIR') #'/projects/my-private-bucket/tmp/mask_test_keep_snow'
    
    args = f"--in_tile_fn {INDEX_FN} \
        --in_tile_layer {INDEX_LYR} \
        --sat_api {SAT_API} \
        --tile_buffer_m 0 \
        --in_tile_num {FOCAL_TILE} \
        --output_dir {OUTDIR} \
        -sy {YEAR_START} -ey {YEAR_STOP} -smd {SEASON_START} -emd {SEASON_STOP} -mc {MAX_CLOUDS} \
        --composite_type {MS_COMP_TYPE} \
        --hls_product {HLS_PRODUCT} \
        --thresh_min_ndvi -1 \
        --min_n_filt_results {MIN_N_FILT_RESULTS} \
        --stat {STAT} \
        --stat_pct {STAT_PCT} \
        --target_spectral_index {TARGET_SPECTRAL}"
    args += " --do_indices"
    #args += " --search_only"
    # args += " --rangelims_red 0.01 0.1" # the default now effectively no limit [-1e9, 1e9]
    args += " --rangelims_red 0.01 1" # the default now effectively no limit [-1e9, 1e9]
    
    ###########################################
    # Output file name
    ###########################################
    fn = f'{OUTDIR}/{MS_COMP_TYPE}_{FOCAL_TILE}_{SEASON_START}_{SEASON_STOP}_{YEAR_START}_{YEAR_STOP}_{STAT}{TARGET_SPECTRAL}.tif'
    if STAT == 'percentile':
        fn = f'{OUTDIR}/{MS_COMP_TYPE}_{FOCAL_TILE}_{SEASON_START}_{SEASON_STOP}_{YEAR_START}_{YEAR_STOP}_{STAT}{STAT_PCT}{TARGET_SPECTRAL}.tif'

    
    fn = Path(fn)

    ###########################################
    # Check if it exists, if it does return 
    if fn.exists():
        print(f'Skipping: {fn} - file already exists')
        return str(fn)

    
    ## 
    # cmd = f'python /projects/my-private-bucket/code-git-shared/icesat2_boreal/lib/build_ms_composite.py {args}'
    # cmd = f'python /projects/my-private-bucket/code-git-shared/icesat2_boreal/lib/build_ms_composite_multip.py {args}'
    cmd = f'python /projects/my-private-bucket/code-git-shared/icesat2_boreal/lib/build_ms_composite_addEVI2.py {args}'
    # !echo $cmd
    !eval $cmd
    ##


    
    #rescaled_multiband_fn = os.path.join(os.path.dirname(fn), os.path.basename(fn).replace('.tif','_rescaled_3band_temp.tif'))
    # plotlib.rescale_multiband_for_plot(fn, rescaled_multiband_fn, bandlist = [5,7,3], pct=[20,90], nodata=-9999.0) 

    
    return str(fn)

In [43]:
# Define input and output paths

####### 90 km - evi2 p95
INDEX_FN = '/projects/my-private-bucket/HLS-1DCNN-AGB/data/shp/atlantic_forest/tiles/br_af_grid90km_prj.gpkg' 
# BASE_OUTDIR = '/projects/my-private-bucket/HLS-1DCNN-AGB/data/tif/HLS_composites/monthly/br_af_grid90km_prj_evi2_max'
BASE_OUTDIR = '/br_af_grid90km_prj_evi2_max' #SAFE
BASE_FINAL_DIR = '/projects/my-private-bucket/HLS-1DCNN-AGB/data/tif/HLS_composites/monthly'
os.makedirs(BASE_OUTDIR, exist_ok=True)
os.makedirs(BASE_FINAL_DIR, exist_ok=True)
############# 60 km evi2 max
# Define input and output paths
# INDEX_FN = '/projects/my-private-bucket/HLS-1DCNN-AGB/data/shp/atlantic_forest/tiles/br_af_grid60km_prj.gpkg' 
# BASE_OUTDIR = '/projects/my-private-bucket/HLS-1DCNN-AGB/data/tif/HLS_composites/monthly/br_af_grid60km_prj_evi2_max'
# BASE_OUTDIR = '/br_af_grid60km_prj_evi2_max'

#######################
### TEST
# INDEX_FN = '/projects/my-private-bucket/HLS-1DCNN-AGB/data/shp/atlantic_forest/tiles/br_af_grid60km_prj.gpkg' 
# BASE_OUTDIR = '/projects/my-private-bucket/HLS-1DCNN-AGB/data/tif/HLS_composites/monthly/br_af_grid60km_prj_ndvi_max_test'



# # Define input and output paths
# INDEX_FN = '/projects/my-private-bucket/HLS-1DCNN-AGB/data/shp/atlantic_forest/tiles/br_af_grid30km_prj.gpkg' 
# BASE_OUTDIR = '/projects/my-private-bucket/HLS-1DCNN-AGB/data/tif/HLS_composites/monthly/br_af_grid30km_prj_evi2_max'


In [44]:
# Set default parameters
SAT_API = 'https://cmr.earthdata.nasa.gov/stac/LPCLOUD'
MS_COMP_TYPE = 'HLS'
HLS_PRODUCT = 'H30'

STAT = 'max'
# STAT = 'percentile'

# TARGET_SPECTRAL = 'ndvi'
TARGET_SPECTRAL = 'evi2'
STAT_PCT = 95.0

MIN_N_FILT_RESULTS = 10
MAX_CLOUDS = 0 # Threshold of max clouds to start search, e.g. when set to 0 it starts at 0% of cloud cover and go up to 90% 


In [45]:
# Get first Layer name (First layer is used as default)
layer_names = fiona.listlayers(INDEX_FN)
# Get the first layer name
first_layer_name = layer_names[0]
INDEX_LYR = first_layer_name

In [46]:
# Define parameters
params = {
    'SAT_API': SAT_API,
    'HLS_PRODUCT': HLS_PRODUCT,
    'MS_COMP_TYPE': MS_COMP_TYPE,
    'MAX_CLOUDS': MAX_CLOUDS,
    'MIN_N_FILT_RESULTS': MIN_N_FILT_RESULTS,
    'STAT': STAT,
    'STAT_PCT': STAT_PCT,
    'TARGET_SPECTRAL': TARGET_SPECTRAL,
    'INDEX_FN': INDEX_FN,
    'INDEX_LYR': INDEX_LYR,
}

In [47]:
# Define year
years = [2022]

# Get all tiles IDs
with fiona.open(INDEX_FN, layer=0) as src:
    tiles = [feature["properties"]["tile_num"] for feature in src]
    
tiles_gpkg = gpd.read_file(INDEX_FN)
tiles = tiles_gpkg["tile_num"].tolist()
# # tiles

In [48]:
len(tiles)

220

In [49]:
#######################################
# Select tiles to run
#######################################
# tiles_run = tiles

# tiles_run = tiles[3:4]
# tiles_run = [89,99]
tiles_run = [89]
# tiles_run


In [50]:
#######################################
# Select based on overlap (doing that during tests if not using 90km grids)
# ref_gpkg_fn = '/projects/my-private-bucket/HLS-1DCNN-AGB/data/shp/atlantic_forest/tiles/br_af_grid90km_prj.gpkg' 
# # ref_gpkg_fn = '/projects/my-private-bucket/HLS-1DCNN-AGB/data/shp/atlantic_forest/tiles/br_af_grid60km_prj.gpkg' 

# ref_tile_num = 89


# gdf_index = gpd.read_file(INDEX_FN)
# gdf_ref = gpd.read_file(ref_gpkg_fn)

# # Select reference tile 89
# ref_tile = gdf_ref[gdf_ref["tile_num"] == ref_tile_num]

# # Spatial join / intersection
# matches = gdf_index[gdf_index.intersects(ref_tile.union_all())]

# # Get the tile_num of matches
# tiles_run = matches["tile_num"].tolist()



In [51]:
tiles_run

# Tile 89 of grid 90 km used as a reference to get GEDI data
# These are the overlapping tiles from grid 60 km
# [176, 177, 178, 196, 197, 198, 210, 211, 212]

[89]

In [52]:
# tiles_run = [210, 211, 212]
# tiles_run = [198]


In [53]:
# import folium
# # Center map on the reference tile
# center = ref_tile.geometry.centroid.iloc[0].coords[0][::-1]  # (lat, lon)

# m = folium.Map(location=center, zoom_start=8)

# # Add reference tile (90 km) in red
# folium.GeoJson(
#     ref_tile,
#     style_function=lambda x: {"color": "red", "weight": 3, "fillOpacity": 0},
#     name="Reference tile (90 km)"
# ).add_to(m)

# # Add matching 30 km tiles in blue
# folium.GeoJson(
#     matches,
#     style_function=lambda x: {"color": "blue", "weight": 2, "fillOpacity": 0.2},
#     tooltip=folium.GeoJsonTooltip(fields=["tile_num"], aliases=["Tile #"])
# ).add_to(m)

# # Add layer control
# folium.LayerControl().add_to(m)

In [54]:
# years = [2022]
# years = [2021]
years = [2020]

# years = [2022, 2021,2020,2019]
# years = [2022,2021,2020,2019,2018]
# years = [2022,2021,2019,2018]

# years = [2019,2018]
# years = [2023]
# years = [2020]

# type(years)

In [55]:
%%time
# Create parameter list and create output directories
# This is set up to do monthly composites
params_list = []
for tile in tiles_run: 
    for year in years:
        # for month in range(2, 4):
        for month in range(1, 13):
            start_day = f"{month:02d}-01"
            end_day = f"{month:02d}-{monthrange(year, month)[1]:02d}"
    
            # output directory for specific run
            # outdir = f'{BASE_OUTDIR}/tile_{tile:03d}/{year}/{month:02d}'
            outdir = f'{BASE_OUTDIR}/tile_{tile:03d}/{year}/'
            
            # os.makedirs(outdir, exist_ok=True)
    
            run_params = copy.deepcopy(params)
            run_params.update({
                'FOCAL_TILE': tile,
                'YEAR': year,
                'SEASON_START': start_day,
                'SEASON_STOP': end_day,
                'OUTDIR': str(outdir)
            })
    
            params_list.append(run_params)



CPU times: user 183 µs, sys: 0 ns, total: 183 µs
Wall time: 188 µs


In [56]:
params_list[-1]['OUTDIR']

'/br_af_grid90km_prj_evi2_max/tile_089/2020/'

In [57]:
len(params_list)

12

In [58]:
# 

In [59]:
# wrapper_composite(params_list[-1])

In [60]:
# params_list[0]#['SEASON_START'].split('-')[0]
# params_list[10:13]

In [61]:
# Test saving in safe directory from
#  Stopped here tile 198, year 2018, month 02
# But running on safe directory from tile 210, 211, 212
# Running 198 all again
# params

In [ ]:
%%time
for params in params_list:
    try:
        print(f"Running tile {params['FOCAL_TILE']}, year {params['YEAR']}, month {params['SEASON_START'].split('-')[0]}")
       
        # Create output directory
        outdir = params['OUTDIR']
        os.makedirs(outdir, exist_ok=True)
        
        # Run composite
        output_file = wrapper_composite(params)
        print(f"Output saved to: {output_file}")


        # # If needed because of disk space
        # # Move from safe dir to end folder
        # final_dir = f'{BASE_FINAL_DIR}{outdir}'
        # # print(outdir)
        # # print(final_dir)
        # # subprocess.run(
        # #     ["rsync", "-av", "--remove-source-files", output_file, final_dir],
        # #     check=True
        # # )
        # # print(f"Moved {output_file} to {final_dir}")

        # try:
        #     subprocess.run(
        #         ["rsync", "-av", "--remove-source-files", output_file, final_dir],
        #         check=True
        #     )
        #     print(f"Moved {output_file} to {final_dir}")
        # except Exception as move_err:
        #     print(f"⚠️ Error moving {output_file} to {final_dir}: {move_err}")
            
        
    except Exception as e:
        print(f"Error processing tile {params['FOCAL_TILE']}, year {params['YEAR']}, month {params['SEASON_START'].split('-')[0]}: {e}")

Running tile 89, year 2020, month 01

Tiles path:		 /projects/my-private-bucket/HLS-1DCNN-AGB/data/shp/atlantic_forest/tiles/br_af_grid90km_prj.gpkg
Tile number:		 89
Output res (m):		 30
in_bbox:		 [5469419.049164498, 7224073.29125729, 5559419.049164498, 7314073.29125729]
bbox 4326:		 [-49.378378833496065, -25.01637488893418, -48.45754764772649, -24.17719601338586]
Getting output dims from buffered (buffer=0.0) original tile geometry...
Output dims:		3000 x 3000
Composite type:		HLS

Getting HLS Surface Reflectance H30 data...
bbox: [-49.378378833496065, -25.01637488893418, -48.45754764772649, -24.17719601338586]

Querying STAC for multispectral imagery...
Catalog title: LPCLOUD STAC Catalog
start date, end date:		 2020-01-01T00:00:00Z 2020-01-31T23:59:59Z

Conducting multispectral image search now ...
Searching for:			['HLSL30_2.0', 'HLSS30_2.0']
Max cloudcover threshold starts at: 0% and won't exceed 90%
Min number of filtered results: 10
partial results (HLSL30_2.0):				0
partial r

In [ ]:
## NOTES

# Monthly composites 1 tile 1 year 60 km
# Wall time: 23min 42s

# Monthly composites 1 tile 1 year 90 km
# Wall time: 45min 2s

# Monthly composites 1 tile 90 km 3 years (2022, 2021,2020)
# Wall time: 2h 16min 58s


In [ ]:
# rsync -av --remove-source-files /br_af_grid90km_prj_evi2_max/ /projects/my-private-bucket/HLS-1DCNN-AGB/data/tif/HLS_composites/monthly/br_af_grid90km_prj_evi2_max/

In [ ]:
# move files when saving them on a safe folder
# rsync -av --remove-source-files /br_af_grid60km_prj_evi2_max/ /projects/my-private-bucket/HLS-1DCNN-AGB/data/tif/HLS_composites/monthly/br_af_grid60km_prj_evi2_max/


# rsync -a → archive mode (preserves permissions, times, symlinks, recursive).

# -v → verbose.

# --remove-source-files → deletes the file in the source once it’s copied.


In [ ]:
# rsync -av --remove-source-files /br_af_grid60km_prj_evi2_p95/ /projects/my-private-bucket/HLS-1DCNN-AGB/data/tif/HLS_composites/monthly/br_af_grid90km_prj_evi2_p95/


## MULTIPROCESSING - Error with multiple requests

In [ ]:
# %%time
# mp.cpu_count() - 1
# with mp.Pool(processes= 5) as pool:
#     fn_list = pool.map(wrapper_composite, params_list)
# with mp.Pool(processes= 12) as pool:
#     fn_list = pool.map(wrapper_composite, params_list)

In [ ]:
# # Plotting to check false composite to check
# fn = fn_list[0]
# fn
# rescaled_multiband_fn = fn.replace('.tif', '_rescaled_3band_temp.tif')
# plotlib.rescale_multiband_for_plot(fn, rescaled_multiband_fn, bandlist = [6,4,3], pct=[20,98], nodata=-9999.0) 

# fig,ax=plt.subplots(figsize=(10,10))
# with rasterio.open(rescaled_multiband_fn) as src:
#     #print(src.profile)
#     show(src.read(),transform=src.transform, ax=ax, title=os.path.basename(rescaled_multiband_fn))

In [ ]:
### Notes

# Time to run 1 tile 1 year 12 processors



In [ ]:
## CHECK ERRORS

## No output is created sometimes when runnning multiple jobs